In [4]:
from langchain_groq import ChatGroq
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain.agents import initialize_agent,AgentType
from langchain_google_community import GoogleSearchAPIWrapper
from langchain_core.tools import Tool
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
GROG_API_KEY = os.getenv("GROQ_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["GOOGLE_CSE_ID"] = GOOGLE_CSE_ID
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "SearchEngineApp"

In [26]:
search = GoogleSearchAPIWrapper()

google_tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func= lambda query: search.results(query, 5),
)

In [27]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki_tool=WikipediaQueryRun(api_wrapper=api_wrapper)

In [28]:
llm=ChatGroq(groq_api_key=GROG_API_KEY,model_name="Llama3-70b-8192",streaming=True)
tools=[google_tool]

search_agent=initialize_agent(tools,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,handling_parsing_errors=True)


In [29]:
question = 'Who is Abdellah Kaissari ?'
response = search_agent.run(question)
print(response)

Abdellah Kaissari is a researcher or scientist in the field of engineering or computer science, possibly focused on plant disease detection or related fields.
